# FALTA FAZER:
1. criterio de selecao
2. algoritmo genetico em si

In [1]:
# import das bibliotecas
import os
import pandas as pd
import random
from greedy_filter import *

In [2]:
# definicao das constantes
PATH_EXCEL = os.getcwd() + '/excel_files/'

Estruturas que irao armazenar os dataframes com o histórico de preços das ações.
1. dict_excels: dicionario onde a chave é o nome da ação e o valor da chave é o dataframe
2. filenames: lista com o nome dos arquivos
3. excels: lista com os dataframes

In [3]:
dict_excels = {}
filenames = []
excels = []
for filename in os.listdir(PATH_EXCEL):
    filenames.append(filename[:filename.find('.')])
    excels.append(pd.read_excel(PATH_EXCEL + filename).sort_values(by=['Exchange Date']).reset_index(drop=True))
    dict_excels[filename[:filename.find('.')]] = (pd.read_excel(PATH_EXCEL + filename).sort_values(by=['Exchange Date']).reset_index(drop=True))

## Modelagem do problema
### Algoritmo Genético 1 - Escolha dos Parâmetros das Regras de Filtro
1. Gene: cada um dos parâmetros do filtro
    1. x - porcentagem de variação: porcentagem acima/abaixo da ultima subida ou queda
       1. varia de 0 a 1
       2. varia de 0.01 a 0.10
    2. h - hold days: após um sinal de compra/venda esperamos por h dias
       1. varia de 1 até o número de dias entre a data atual e a data onde acabam as informacoes
       2. varia de 1 até 30
    3. d - delay days: após receber um sinal ignora-se os próximos d dias
       1. varia de 1 até o número de dias entre a data atual e a data onde acabam as informacoes
       2. varia de 1 até 30
    4. p - previous days: olha-se p dias para tomar uma decisão
       1. varia de 0 até o número de dias transcorridos até o momento
       2. varia de 30 até o número máximo de dias
2. Cromossomo: conjunto de parâmetros
    1. Estrutura usada: lista com os parâmetros &rarr; [x, h, d, p]
3. População: conjunto de cromossomos
    1. Tamanho da População:
4. Mutação:
    1. taxa de mutação:
5. Crossover:
    1. taxa de crossover:
6. Critério de Parada:
7. Seleção:
    1. Fitness: cálculo do lucro obtido por cada cromossomo
    2. Técnica de Seleção:
        1. Técnica 1:
        2. Técnica 2:

Funções para realizar o crossover entre dois cromossomos

In [4]:
def crossover(chromosome1, chromosome2):
    """
        Funcao para realizar o crossover entre dois cromossomos
        :param: chromosome1 - cromossomo pai
        :param: chromosome2 - cromossomo mae
        :return: new_chromosome1 - primeiro filho gerado no crossover
        :return: new_chromosome2 - segundo filho gerado no crossover
    """
    # escolhe um gene aleatorio para realizar o crossover
    rand = random.randint(1,len(chromosome1) - 1)

    # gera os dois novos cromossomos
    new_chromosome1 = chromosome1[:rand] + chromosome2[rand:]
    new_chromosome2 = chromosome2[:rand] + chromosome1[rand:]
    
    return new_chromosome1, new_chromosome2

Funções para realizar a mutação de um cromossomo

In [5]:
def mutation(chromossome, today_index, last_day_index):
    """
        Funcao para realizar a mutacao de um dado cromossomo
        :param: chromossome - cromossomo que ira passar pelo processo de mutacao
        :param: today_index - indice da tabela referente a data atual da acao
        :param: last_day_index - indice da tabela referente a ultima data da qual se tem informacao da acao
        :return: new_chromossome - novo cromossomo apos o processo de mutacao
    """
    # faz a copia do cromossomo original
    new_chromossome = chromossome
    
    # escolhe, aleatoriamente, um gene para ser alterado
    gene_position = random.randint(0,len(chromossome) - 1)
    
    # modifica o gene escolhido seguindo, como unica regra, os valores que ele pode assumir
    if gene_position == 0:
        new_chromossome[0] = random.random()
    elif gene_position == 1:
        new_chromossome[1] = random.randint(0, last_day_index - today_index)
    elif gene_position == 2:
        new_chromossome[2] = random.randint(0, last_day_index - today_index)
    elif gene_position == 3:
        new_chromossome[3] = random.randint(0, today_index)
    else:
        raise Exception('Gene inexistente no cromossomo!')
        
    return new_chromossome

def mutation_v2(chromossome):
    """
        Funcao para realizar a mutacao de um dado cromossomo
        :param: chromossome - cromossomo que ira passar pelo processo de mutacao
        :return: new_chromossome - novo cromossomo apos o processo de mutacao
    """
    # faz a copia do cromossomo original
    new_chromossome = chromossome
    
    # escolhe, aleatoriamente, um gene para ser alterado
    gene_position = random.randint(0,len(chromossome) - 1)
    
    # modifica o gene escolhido seguindo, como unica regra, os valores que ele pode assumir
    if gene_position == 0:
        new_chromossome[0] = random.uniform(0.01,0.10)
    elif gene_position == 1:
        new_chromossome[1] = random.randint(1,30)
    elif gene_position == 2:
        new_chromossome[2] = random.randint(1,30)
    elif gene_position == 3:
        new_chromossome[3] = random.randint(30,500)
    else:
        raise Exception('Gene inexistente no cromossomo!')
        
    return new_chromossome

Função para realizar a inicialização randômica de uma população

In [6]:
def create_population(population_size):
    """
        Funcao para criar uma populacao randomica de cromossomos
        :param: population_size - tamanho da populacao que sera criada
        :return: population - nova populacao
    """
    population = []
    for i in range(0, population_size):
        population.append([random.random(), random.randint(1,30), random.randint(1,30), random.randint(1,500)])
        
    return population

In [13]:
def fitness(np_array, chromossomes, budget):
    """
        Funcao para calcular o lucro de cada um dos cromossomos de uma dada populacao
        :param: np_array - conjunto de dados de determinada acao
        :param: chromossomes - lista de cromossomos que sera avaliada
        :param: budget - dinheiro inicial do problema
        :return: fit_chromossomes - matriz com os cromossomos e o lucro(em porcentagem) obtido por eles
    """
    fit_chromossomes = []
    for chromossome in chromossomes:
        money = greedy_filter_rule(np_array, chromossome, budget)
        fit_chromossomes.append([chromossome, (money-budget)/budget])
        
    return fit_chromossomes

In [14]:
np_array = excels[0].values
chromossomes = create_population(10)
budget = 10000
fitness(np_array, chromossomes, budget)

[[[0.003586974437413959, 9, 26, 398], 1.170062999999999],
 [[0.9873071062401202, 24, 20, 75], 0.0],
 [[0.260349698576539, 28, 1, 485], 1.4162860000000002],
 [[0.8459171881349796, 10, 27, 28], 0.0],
 [[0.16680953853788993, 30, 21, 72], 0.48962199999999995],
 [[0.4425677313540157, 4, 3, 132], 0.0],
 [[0.4811693745131408, 28, 6, 198], 0.0],
 [[0.7974728689933931, 19, 10, 137], 0.0],
 [[0.7369850174651227, 25, 18, 309], 0.0],
 [[0.635594239644741, 1, 6, 31], 0.0]]

In [37]:
number_to_combine = 12 # número "n" que serão combinados 2 a 2 para gerar a nova geração
len_next_generation = 100 # tamanho que definimos para a proxima geração
percentage_new_generation = 0.9 # porcentagem máxima da nova geracao que será por crossover

new_generation = []
for i in range (number_to_combine):
    for j in range (i+1,number_to_combine):
        new_chromossome1, new_chromossome2 = crossover(chromossomes[i], chromossomes[j])
        new_generation.append(new_chromossome1)
        new_generation.append(new_chromossome2)

actual_len = len(new_generation)
if (actual_len > len_next_generation * percentage_new_generation):
    new_generation = new_generation[:int(len_next_generation * percentage_new_generation)]
    actual_len = len(new_generation)

for i in range (0, len_next_generation - actual_len):
    new_generation.append(chromossomes[i])
        
print (len(new_generation))
fitness(np_array, new_generation, budget)

100


[[[0.6545666149176183, 21, 18, 167], 0.0],
 [[0.9409669790405667, 9, 5, 185], 0.0],
 [[0.6545666149176183, 21, 16, 355], 0.0],
 [[0.2947182397067649, 27, 5, 185], 0.0],
 [[0.6545666149176183, 21, 18, 282], 0.0],
 [[0.4391587897794711, 21, 5, 185], 0.0],
 [[0.6545666149176183, 21, 13, 122], 0.0],
 [[0.44040325229043376, 15, 5, 185], 0.0],
 [[0.6545666149176183, 21, 5, 131], 0.0],
 [[0.48625978382466184, 25, 25, 185], 0.0],
 [[0.6545666149176183, 21, 15, 20], 0.0],
 [[0.6322985067781749, 15, 5, 185], 0.0],
 [[0.6545666149176183, 9, 21, 347], 0.0],
 [[0.8999324276282289, 21, 5, 185], 0.0],
 [[0.6545666149176183, 21, 5, 217], 0.0],
 [[0.7741343449598641, 2, 10, 185], 0.0],
 [[0.6545666149176183, 21, 5, 59], 0.0],
 [[0.19030238687979273, 29, 25, 185], 0.0],
 [[0.6545666149176183, 21, 21, 385], 0.0],
 [[0.37589729336978017, 21, 5, 185], 0.0],
 [[0.6545666149176183, 4, 25, 222], 0.0],
 [[0.021905360164286458, 21, 5, 185], 0.19994000000000034],
 [[0.9409669790405667, 9, 16, 355], 0.0],
 [[0.29

In [14]:
print (chromossomes)

[[0.6545666149176183, 21, 5, 185], [0.9409669790405667, 9, 18, 167], [0.2947182397067649, 27, 16, 355], [0.4391587897794711, 21, 18, 282], [0.44040325229043376, 15, 13, 122], [0.48625978382466184, 25, 25, 131], [0.6322985067781749, 15, 15, 20], [0.8999324276282289, 9, 21, 347], [0.7741343449598641, 2, 10, 217], [0.19030238687979273, 29, 25, 59], [0.37589729336978017, 21, 21, 385], [0.021905360164286458, 4, 25, 222], [0.9400888363580845, 5, 13, 100], [0.36647763751693463, 12, 2, 155], [0.9429109033364939, 11, 2, 96], [0.17229671445501016, 9, 24, 381], [0.7629052455182017, 23, 2, 317], [0.889025651302524, 5, 17, 423], [0.416874565617806, 2, 8, 399], [0.33686422595634136, 1, 30, 445], [0.251002331755819, 28, 5, 437], [0.2869569758851811, 8, 27, 408], [0.4162287649056645, 26, 6, 396], [0.17535234171617853, 15, 21, 170], [0.08581197284442654, 29, 25, 441], [0.5975961302401221, 3, 19, 33], [0.4790226453365256, 19, 21, 193], [0.3115130796554427, 2, 10, 91], [0.5242249197711314, 19, 21, 169], 